In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [13]:
class Plane:
    def __init__(self,_id,_pos,_vel):
        self.id = _id
        self.pos = _pos #Not sure if we want this defined as np array or as pos_x, pos_y and pos_z
        self.vel = _vel #Not sure if we want this defined as np array or as vel_x, vel_y and vel_z

    def __str__(self):
        return "ID: {id}, Pos: {pos}, Vel: {vel}".format(id = self.id, pos = self.pos, vel = self.vel)

In [22]:
def create_n_planes(n_planes,max_pos,max_vel):
    # n_planes controls how many planes are created
    # max_pos is used as size of grid planes put randomly in (just temp)
    # max_vel scales the maximum velocity of the planes
    out = []
    for id in range(n_planes):
        random_pos = np.random.rand(3) * max_pos
        random_vel = np.random.rand(3) * max_vel
        out.append(Plane(id,random_pos,random_vel))
    return out

In [23]:
planes = create_n_planes(5,10,1)
for plane in planes:
    print(plane)

ID: 0, Pos: [8.62087726 2.0460121  3.8668769 ], Vel: [0.59989352 0.63488617 0.72026095]
ID: 1, Pos: [7.08327567 6.75924416 1.21285412], Vel: [0.42417607 0.79582141 0.89333617]
ID: 2, Pos: [2.21053893 8.50901098 1.11278897], Vel: [0.89328997 0.91315226 0.68409994]
ID: 3, Pos: [6.64908983 4.56862807 1.80701945], Vel: [0.8152133  0.98351424 0.64495258]
ID: 4, Pos: [6.90031439 4.1284072  8.54440562], Vel: [0.64370989 0.89357826 0.09688858]
